In [ ]:
import os
import sys
import subprocess

print("🔄 Installing clean libraries...")

# 1. Uninstall broken packages
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "numpy", "opencv-python-headless", "ultralytics"], stdout=subprocess.DEVNULL)

# 2. Install compatible versions (Pinned for stability)
# numpy<2.0 is CRITICAL to prevent the error you just saw
install_cmd = [
    sys.executable, "-m", "pip", "install", 
    "numpy<2.0", 
    "ultralytics", 
    "onnxruntime-gpu",
    "opencv-python-headless", # Ensures CV2 works
    "--no-cache-dir"
]
subprocess.run(install_cmd, stdout=subprocess.DEVNULL)

# 3. Reload environment
import site
import importlib
importlib.reload(site)

print("✅ Installation Complete. You can now run the tracker.")

In [ ]:
import sys
import subprocess

print("⬇️ Installing ALL required modules at once...")

# This list covers every dependency boxmot needs to avoid future errors
requirements = [
    "filterpy",          # The error you just saw
    "lapx",              # Linear Assignment
    "yacs",              # Config helper
    "ftfy",              # Text fixer
    "regex",             # Text helper
    "loguru",            # Logging
    "thop",              # Performance counter
    "scikit-image",      # Image processing
    "gdown",             # File downloader
    "onnxruntime-gpu",   # Model acceleration
    "ultralytics",       # YOLO
    "numpy<2.0"          # Critical version lock
]

# Install everything in one command
subprocess.run([sys.executable, "-m", "pip", "install"] + requirements, stdout=subprocess.DEVNULL)

print("✅ ALL MODULES INSTALLED. The code below will work now.")

In [ ]:
import sys
import subprocess
import os

print("⬇️ FIXING BOXMOT VERSION...")

# 1. Uninstall the broken/random version
subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "boxmot"], stdout=subprocess.DEVNULL)

# 2. Install specific stable version (10.0.43) + Dependencies
# We install the dependencies manually first to avoid conflicts
deps = ["filterpy", "lapx", "yacs", "ftfy", "regex", "loguru", "thop", "termcolor", "scikit-image", "gdown"]
subprocess.run([sys.executable, "-m", "pip", "install"] + deps, stdout=subprocess.DEVNULL)

# 3. Install the specific BoxMOT version
subprocess.run([sys.executable, "-m", "pip", "install", "boxmot==10.0.43", "--no-deps"], stdout=subprocess.DEVNULL)

print("✅ Version Fixed. Now run the main code.")

In [ ]:
import cv2
import numpy as np
import onnxruntime as ort
from ultralytics import YOLO
from pathlib import Path
import torch
import os
import sys

# Import Deep OC-SORT
try:
    from boxmot import DeepOCSORT
except ImportError:
    from boxmot.trackers.deepocsort.deep_oc_sort import DeepOCSORT

# ==========================================
# 1. DEBUG SETTINGS (High Sensitivity)
# ==========================================
CONF_THRESH = 0.20        # Lower confidence to see more
ASPECT_RATIO_MAX = 0.9
MIN_AREA = 2000

# -- DEBUG TUNING --
NECK_ZONE_SIZE = 0.20     # Wider zone (0.20 instead of 0.15)
MIN_SNATCH_FRAMES = 1     # 🚨 INSTANT TRIGGER (For testing)
MAX_SNATCH_FRAMES = 100   

# ==========================================
# 2. SETUP
# ==========================================
tracker = DeepOCSORT(
    model_weights=Path('osnet_x0_25_msmt17.pt'),
    device='cuda' if torch.cuda.is_available() else 'cpu',
    fp16=False,
    det_thresh=CONF_THRESH
)

def preprocess_pose(img):
    resized = cv2.resize(img, (256, 256))
    mean, std = np.array([123.675, 116.28, 103.53]), np.array([58.395, 57.12, 57.375])
    return np.expand_dims(((cv2.cvtColor(resized, cv2.COLOR_BGR2RGB) - mean) / std).astype(np.float32).transpose(2, 0, 1), axis=0)

def get_kpts(outputs, h, w):
    simcc_x, simcc_y = outputs[0], outputs[1]
    x = np.argmax(simcc_x, axis=2)[0] / simcc_x.shape[2] * w
    y = np.argmax(simcc_y, axis=2)[0] / simcc_y.shape[2] * h
    return np.stack([x, y], axis=-1)

# ==========================================
# 3. MAIN LOOP
# ==========================================
YOLO_PATH = '/kaggle/input/best-yolov8m/pytorch/default/1/best _yolov8m.pt'
RTMPOSE_ONNX_PATH = '/kaggle/input/rtm-pose/onnx/default/1/rtmpose-m_ap10k_256.onnx' 
VIDEO_PATH = '/kaggle/input/snatching-videos/snatching_videos/snatching_18.mp4'
OUTPUT_PATH = '/kaggle/working/debug_output.mp4'

if not os.path.exists(VIDEO_PATH):
    print("❌ Video missing.")
    sys.exit()

try:
    yolo = YOLO(YOLO_PATH)
except:
    yolo = YOLO('yolo11s.pt')

ort_sess = ort.InferenceSession(RTMPOSE_ONNX_PATH, providers=['CPUExecutionProvider'])
in_name = ort_sess.get_inputs()[0].name

cap = cv2.VideoCapture(VIDEO_PATH)
W, H = int(cap.get(3)), int(cap.get(4))
FPS = cap.get(cv2.CAP_PROP_FPS) or 25
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), int(FPS), (W, H))

interaction_timers = {}

print("🚀 Running DEBUG MODE (X-Ray)...")

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    frame_count += 1
    if frame_count % 20 == 0: print(f"Processing frame {frame_count}...", end='\r')
    
    # 1. DETECT
    results = yolo.predict(frame, conf=CONF_THRESH, verbose=False, classes=[0])
    dets_to_track = []
    
    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss = r.boxes.cls.cpu().numpy()
        for box, conf, cls in zip(boxes, confs, clss):
            x1, y1, x2, y2 = map(int, box[:4])
            w, h = (x2-x1), (y2-y1)
            if (w*h) < MIN_AREA: continue
            if (w / h) > ASPECT_RATIO_MAX: continue
            dets_to_track.append([x1, y1, x2, y2, conf, cls])
    
    dets_to_track = np.array(dets_to_track)
    tracks = tracker.update(dets_to_track, frame) if len(dets_to_track) > 0 else []
    
    # 2. COLLECT DATA
    people = []
    for track in tracks:
        x1, y1, x2, y2 = map(int, track[:4])
        tid = int(track[4])
        
        # ZONE GEOMETRY
        neck_cx = int(x1 + (x2-x1)//2)
        neck_cy = int(y1 + (y2-y1)*0.15)
        neck_rad = int((y2-y1)*NECK_ZONE_SIZE)
        
        hands = []
        shoulders = []
        crop = frame[max(0,y1):min(H,y2), max(0,x1):min(W,x2)]
        if crop.size > 0:
            pose_in = preprocess_pose(crop)
            kpts = get_kpts(ort_sess.run(None, {in_name: pose_in}), crop.shape[0], crop.shape[1])
            
            # Draw Skeleton for Debugging
            for k in [5,6,7,8,9,10]: # Shoulders, Elbows, Wrists
                if k < len(kpts):
                    px, py = int(kpts[k][0]+x1), int(kpts[k][1]+y1)
                    cv2.circle(frame, (px, py), 3, (0, 255, 255), -1) # Yellow dots
            
            # Store Hands (9, 10)
            for k in [9, 10]:
                if k < len(kpts):
                    hands.append((int(kpts[k][0]+x1), int(kpts[k][1]+y1)))
                    
        people.append({'id': tid, 'box': (x1,y1,x2,y2), 'neck': (neck_cx,neck_cy,neck_rad), 'hands': hands})

    # 3. INTERACTION CHECK
    snatch_alerts = set()
    
    for p1 in people: # Thief?
        for p2 in people: # Victim?
            if p1['id'] == p2['id']: continue
            
            pair_key = (p1['id'], p2['id'])
            vx, vy, vrad = p2['neck']
            
            # DEBUG: Draw Neck Zone on everyone (Blue Circle)
            cv2.circle(frame, (vx, vy), vrad, (255, 0, 0), 1)
            
            is_touching = False
            min_dist = 9999
            
            for hx, hy in p1['hands']:
                # DEBUG: Draw large Red Dot on Hands
                cv2.circle(frame, (hx, hy), 5, (0, 0, 255), -1)
                
                dist = np.sqrt((hx-vx)**2 + (hy-vy)**2)
                if dist < min_dist: min_dist = dist
                
                if dist < vrad:
                    is_touching = True
                    # Draw Line showing the contact
                    cv2.line(frame, (hx, hy), (vx, vy), (0, 0, 255), 2)
                    cv2.putText(frame, f"{int(dist)}px", (hx, hy-10), 0, 0.5, (0,255,255), 1)

            if is_touching:
                if pair_key not in interaction_timers: interaction_timers[pair_key] = 0
                interaction_timers[pair_key] += 1
                
                frames = interaction_timers[pair_key]
                if frames >= MIN_SNATCH_FRAMES and frames <= MAX_SNATCH_FRAMES:
                    snatch_alerts.add(p1['id'])
                    snatch_alerts.add(p2['id'])
            else:
                if pair_key in interaction_timers:
                    interaction_timers[pair_key] = max(0, interaction_timers[pair_key] - 1)
                    if interaction_timers[pair_key] == 0: del interaction_timers[pair_key]

    # 4. DRAW
    for p in people:
        tid = p['id']
        x1, y1, x2, y2 = p['box']
        
        color = (0, 255, 0)
        if tid in snatch_alerts:
            color = (0, 0, 255)
            cv2.putText(frame, "SNATCH DETECTED!", (x1, y1-30), 0, 0.8, color, 2)
            nx, ny, nr = p['neck']
            cv2.circle(frame, (nx, ny), nr, color, 3) # Thick Red Circle
            
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"ID:{tid}", (x1, y1-5), 0, 0.6, (255,255,0), 2)

    out.write(frame)

cap.release()
out.release()
print(f"✅ DONE: {OUTPUT_PATH}")